In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
import pandas as pd
import os
import sys
sys.path.insert(1, 'scripts')
import torch

## Save prediction outputs

In [2]:
## load model
model_checkpoint_path = {
    'claims': 'model/for_claims/checkpoint/train-300-t1',
    'premises': 'model/for_premises/checkpoint/train-300-p1'
}
scenario = ['auto-driving', 'bitcoin-invest']


In [3]:
for task, model_path in model_checkpoint_path.items():
    config = AutoConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    for sc in scenario:
        data = pd.read_csv(os.path.join('data', task, f"evaluation-{sc}.csv"))
        inputs = tokenizer(data.text.to_list(), max_length=300,
                           return_tensors='pt', padding=True, truncation=True)
        summary_ids = model.generate(inputs["input_ids"])
        output = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
        df = pd.DataFrame({
            'text': data.text.tolist(),
            'ground-truth': data[task[:-1]].tolist(),
            'prediction': output
        })
        df.to_csv(os.path.join('data/prediction_results', f'prediction-results-{sc}-{task}.csv'), index=False)
    del model

/home/dingdong/anaconda3/envs/transformers/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
2022-08-27 14:27:21.873102: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
# from load_data import find_xml_files, load_objects, load_data
# data = load_data()